In [58]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

df = pd.read_csv('gaze_positions.csv')
panorama = cv2.imread('panorama.png')

# Initiate SIFT detector
sift = cv2.SIFT_create()

FLANN_INDEX_KDTREE = 1
MIN_MATCH_COUNT = 10
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
kp2, des2 = sift.detectAndCompute(panorama, None)

vidcap = cv2.VideoCapture('world.mp4')
success, frame = vidcap.read()

video_fps = vidcap.get(cv2.CAP_PROP_FPS)
total_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)
height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))

df['X'] = df['norm_pos_x'] * width
df['Y'] = height - df['norm_pos_y'] * height
df = df[['world_index', 'X', 'Y']]

df = df.astype({'world_index': int, 'X': int, 'Y': int})

In [59]:
writer = cv2.VideoWriter(f"output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), video_fps, (width + panorama.shape[1], height))
frame_idx = 0

while success:

    img2 = panorama

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(frame, None)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    if len(good)> MIN_MATCH_COUNT:
        src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
        dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        matchesMask = mask.ravel().tolist()
        
        h,w,c = frame.shape
        pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
        dst = cv2.perspectiveTransform(pts, M)
        img2 = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)

        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                    singlePointColor = None,
                    matchesMask = matchesMask, # draw only inliers
                    flags = 3)
        
        # if frame_idx == 0:
        #     result = cv2.drawMatches(frame, kp1, img2, kp2, good, None, **draw_params)

        # else:
        #     draw_params = dict(matchColor = (0,255,0), # draw matches in green color
        #     singlePointColor = None,
        #     matchesMask = matchesMask, # draw only inliers
        #     flags = 3)

        #     result = np.empty(result.shape)
        #     result = cv2.drawMatches(frame, kp1, img2, kp2, good, result, **draw_params)

        result = cv2.drawMatches(frame, kp1, img2, kp2, good, None, **draw_params)

        writer.write(result)

    else:
        matchesMask = None



    success, frame = vidcap.read()
    frame_idx += 1

    print(f"\rDone {round((frame_idx / total_frames) * 100)}%", end='')

writer.release()
vidcap.release()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\features2d\src\draw.cpp:156: error: (-201:Incorrect size of input array) outImg has size less than need to draw img1 and img2 together in function 'cv::_prepareImgAndDrawKeypoints'


In [ ]:
cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS + cv2.DRAW_MATCHES_FLAGS_DRAW_OVER_OUTIMG

3